<a href="https://colab.research.google.com/github/bominwang/inviscid-viscid-flow-correct-neural-network/blob/HB-2-surface-pressure/HB2_SurfacePressurePredicate_NNAS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
import torch
import random
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader, TensorDataset

In [26]:
!pip install optuna
import optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [27]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [28]:
def building_dataset(ins_path, lab_path, train_volume, device):
  ins = pd.read_csv(ins_path)
  labs = pd.read_csv(lab_path)
  
  ins = np.array(ins)
  labs = np.array(labs)
  labs = labs[:,1].reshape(-1,10)

  data_volume = ins.shape[0]
  test_volume = data_volume - train_volume
  
  data_index = list(range(data_volume))
  train_index = random.sample(range(0, data_volume), train_volume)
  test_index = np.setdiff1d(data_index, train_index)

  train_data_ins = torch.from_numpy(ins[train_index, :])
  train_data_labs = torch.from_numpy(labs[train_index, :])
  
  test_data_ins = torch.from_numpy(ins[test_index, :])
  test_data_labs = torch.from_numpy(labs[test_index, :])
  
  train_set = TensorDataset(train_data_ins, train_data_labs)
  train_set = DataLoader(train_set, batch_size = train_volume, shuffle = True)
  
  test_set = TensorDataset(test_data_ins, train_data_labs)
  test_set = DataLoader(test_set, batch_size = test_volume, shuffle = True)

  return train_set, test_set

In [29]:
def relative_root_mean_squared_error(pred, true):
  size = pred.size(dim = 1)
  error = torch.zeros(size, 1)
  normalization_factor = torch.zeros(size, 1)
  for index in range(size):
    pred_element = pred[index, :]
    true_element = true[index, :]
    error[index, :] = torch.dot(pred_element - true_element, pred_element - true_element)
    normalization_factor[index,:] = torch.dot(true_element, true_element)
  return torch.sqrt(torch.div(torch.sum(error),torch.sum(normalization_factor)))

In [30]:
def define_inviscid_nn(hyperparameter):
  num_layers = hyperparameter.suggest_int('num_layers', 1, 4)
  layer = []
  in_features = 2
  # hidden layer
  for i in range(num_layers):
    out_features = hyperparameter.suggest_int(f'n_neurons_{i}', 10, 400)
    layer.append(nn.Linear(in_features, out_features))
    layer.append(nn.ReLU())
    p = hyperparameter.suggest_float(f'dropout_{i}', 0.2, 0.5)
    layer.append(nn.Dropout(p))
    in_features = out_features
  # output layer
  layer.append(nn.Linear(in_features, 10))
  
  return nn.Sequential(*layer)

In [33]:
def objective_function(hyperparameter):
  model = define_inviscid_nn(hyperparameter).to(device)
  
  lr = hyperparameter.suggest_float('lr',1e-5, 1e-1, log = True)
  optimizer = optim.SGD(model.parameters(), lr = lr)
  loss_function = nn.MSELoss()
  
  ins_path = 'drive/MyDrive/Colab Notebooks/bm_wang/data/HB2/inv/inv_input_data.csv'
  labs_path = 'drive/MyDrive/Colab Notebooks/bm_wang/data/HB2/inv/inv_output_data.csv'
  
  train_set, test_set = building_dataset(ins_path, labs_path, 100, device)
  
  epoches = 1000
  for epoch in range(epoches):
    for index, (ins, lab) in enumerate(train_set):
      model.train()
      ins = Variable(ins).reshape(-1,2).to(torch.float32).to(device)
      lab = Variable(lab).reshape(-1,10).to(torch.float32).to(device)
      optimizer.zero_grad()
      pred = model(ins)
      rmse = loss_function(pred, lab)
      rmse.backward()
      optimizer.step()

    model.eval()
    with torch.no_grad():
      for index, (ins, lab) in enumerate(test_set):
        ins = ins.reshape(-1,2).to(torch.float32).to(device)
        lab = lab.reshape(-1,10).to(torch.float32).to(device)
        pred = model(ins)
        rrmse = relative_root_mean_squared_error(pred, lab)
    
    hyperparameter.report(rrmse, epoch)
  return rrmse

In [34]:
study = optuna.create_study(study_name = 'inviscid_nn', direction='minimize')
study.optimize(objective_function, n_trials = 100, timeout = 600)

pruned_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED]
complete_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]

[I 2023-02-08 10:38:04,139] A new study created in memory with name: inviscid_nn
[I 2023-02-08 10:38:07,804] Trial 0 finished with value: 0.052408766001462936 and parameters: {'num_layers': 1, 'n_neurons_0': 102, 'dropout_0': 0.4917841514430039, 'lr': 0.09198190929798847}. Best is trial 0 with value: 0.052408766001462936.
[I 2023-02-08 10:38:16,464] Trial 1 finished with value: 0.4131770133972168 and parameters: {'num_layers': 4, 'n_neurons_0': 291, 'dropout_0': 0.21669631255418403, 'n_neurons_1': 210, 'dropout_1': 0.47840774356329624, 'n_neurons_2': 193, 'dropout_2': 0.41776091285323325, 'n_neurons_3': 117, 'dropout_3': 0.49199362225769033, 'lr': 0.0029694369186513765}. Best is trial 0 with value: 0.052408766001462936.
[I 2023-02-08 10:38:21,444] Trial 2 finished with value: 0.5867802500724792 and parameters: {'num_layers': 3, 'n_neurons_0': 378, 'dropout_0': 0.2397238565860198, 'n_neurons_1': 34, 'dropout_1': 0.23721166569317556, 'n_neurons_2': 43, 'dropout_2': 0.30213175898710287, '